In [2]:
%run Requirements.ipynb

24/05/07 13:05:35 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
24/05/07 13:05:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/07 13:05:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Consolidated Race Results With Drivers and Constructors

In [88]:
df = spark.sql(
    """WITH RaceDriverStandings AS (
    SELECT
        CAST(R.raceid AS INT) AS RaceID,
        CAST(R.year AS INT) AS Year,
        CAST(R.round AS INT) AS Round,
        CAST(R.circuitid AS INT) AS CircuitID,
        CAST(R.name AS STRING) AS RaceName,
        CAST(R.date AS DATE) AS RaceDate,
        CAST(R.time AS STRING) AS RaceTime,
        CAST(DS.driverid AS INT) AS DriverID,
        CAST(DS.points AS INT) AS DriverPoints,
        CAST(DS.position AS INT) AS DriverPosition
    FROM
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/races` AS R
    INNER JOIN
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/driver_standings` AS DS ON R.raceId = DS.raceId
),
RaceConstructorStandings AS (
    SELECT
        CAST(R.raceid AS INT) AS RaceID,
        CAST(CS.constructorId AS INT) AS ConstructorID,
        CAST(CS.points AS INT) AS ConstructorPoints,
        CAST(CS.position AS INT) AS ConstructorPosition
    FROM
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/races` AS R
    INNER JOIN
        PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructor_standings` AS CS ON R.raceId = CS.raceId
)
SELECT
    R.RaceID,
    R.Year,
    R.Round,
    R.CircuitID,
    R.RaceName,
    R.RaceDate,
    R.RaceTime,
    R.DriverID,
    R.DriverPoints,
    R.DriverPosition,
    CS.ConstructorID,
    CS.ConstructorPoints,
    CS.ConstructorPosition,
    RS.Laps,
    RS.Grid,
    RS.FastestLap,
    RS.FastestLapTime,
    RS.FastestLapSpeed,
    RS.StatusID,
    CONCAT(R.RaceID,'-',R.DriverID) AS RaceDriverID,
    CONCAT(R.RaceID,'-',R.DriverID,'-',CS.ConstructorID) AS RaceDriverConstructorID
FROM
    RaceDriverStandings AS R
INNER JOIN
    RaceConstructorStandings AS CS ON R.RaceID = CS.RaceID
INNER JOIN
    PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/results` AS RS ON R.RaceID = RS.RaceID AND R.DriverID = RS.DriverID AND CS.ConstructorID = RS.ConstructorID"""
)

df.write.mode("overwrite").format('parquet').options(header=True).save('MartData/TotalRaceResults')

Driver Details

In [51]:
df = spark.sql(
    """SELECT 
    CAST(driverid AS INT) AS DriverID,
    CAST(CONCAT(forename,' ',surname) AS STRING) AS DriverName,
    CAST(dob AS DATE) AS DateOfBirth,
    CAST(nationality AS STRING) AS Nationality
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/drivers`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/DriverDetails')

Constructor Details

In [55]:
df = spark.sql(
    """SELECT 
        CAST(constructorid AS INT) AS ConstructorID,
        CAST(name AS STRING) AS ConstructorName,
        CAST(nationality AS STRING) AS Nationality
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructors`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/ConstructorDetails')

Circuits Details

In [58]:
df = spark.sql(
    """SELECT 
        CAST(circuitid AS INT) AS CircuitID,
        CAST(name AS STRING) AS CircuitName,
        CAST(location AS STRING) AS CircuitLocation,
        CAST(country AS STRING) AS Country
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/circuits`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/CircuitDetails')

Qualifying Details

In [92]:
df = spark.sql(
    """SELECT 
        CAST(qualifyid AS INT) AS QualifyID,
        CAST(raceid AS INT) AS RaceID,
        CAST(driverid AS INT) AS DriverID,
        CAST(constructorid AS INT) AS ConstructorID,
        CAST(position AS INT) AS Position,
        CAST(q1 AS STRING) AS Q1,
        CAST(q2 AS STRING) AS Q2,
        CAST(q3 AS STRING) AS Q3,
        CONCAT(RaceID,'-',DriverID,'-',ConstructorID) AS RaceDriverConstructorID
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/qualifying`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/QualifyingDetails')

Lap Time Details

In [10]:
lap_times_df = spark.read.parquet('/Users/ronitguptaaa/Documents/FormulaOne/StagingData/lap_times', header=True)

total_time_df = lap_times_df.groupBy("driverid", "raceid") \
    .agg(sum("milliseconds").alias("total_milliseconds"))

total_time_df = total_time_df.withColumn("total_seconds", total_time_df["total_milliseconds"] / 1000)

total_time_df = total_time_df.withColumn("hours", total_time_df["total_seconds"] / 3600) \
    .withColumn("minutes", (total_time_df["total_seconds"] % 3600) / 60) \
    .withColumn("seconds", total_time_df["total_seconds"] % 60)

total_time_df = total_time_df.withColumn("total_time", expr("concat_ws(':', lpad(int(hours), 2, '0'), lpad(int(minutes), 2, '0'), lpad(int(seconds), 2, '0'))"))

total_time_df = total_time_df.drop("total_milliseconds", "total_seconds", "hours", "minutes", "seconds")

result_df = lap_times_df.join(total_time_df, ["driverid", "raceid"], "left")

result_df = result_df.select("raceid", "driverid", "total_time")
result_df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/LapTimeData')
